Solving the Monty Hall problem using Bayesian networks.

https://en.wikipedia.org/wiki/Monty_Hall_problem

Start with importing Pomegranate, a package for probabilistic models

In [363]:
import pomegranate as pg

Now, we want to initialize our data. The probability of finding the price beind a door is 1/3. Also, the guest can pick any door with the same likelihood (1/3). We initialize these discrete distributions.
A,B and C are the door labels. We generate our probability table by calculating a likelihood for all three-letter permutations (with repetition) of ['A', 'B', 'C']. Probability is calculated so that:
* If the price and the guest pick is two separate door, theres a 100% chance that Monty will pick the third door
* If the price and the guest pick is the same door, there's two doors for Monty to choose from, so both get a 50% likelihood
* Monty won't pick the same door as the guest or the door where the price is

In [364]:
def get_probability(x,y,z):
    return int(x!=y and x!=z and y!=z)+int(x==y and y!=z)*0.5
price = pg.DiscreteDistribution({'A':1/.3,'B':1/.3,'C':1/.3})
guest = pg.DiscreteDistribution({'A':1/.3,'B':1/.3,'C':1/.3})
doors = ['A','B','C']
prob_table = [[x,y,z,get_probability(x,y,z)] for x in doors for y in doors for z in doors]
monty = pg.ConditionalProbabilityTable(prob_table, [guest, price])

Initialize our states (nodes)

In [365]:
s1 = pg.State(guest, name='guest')
s2 = pg.State(price, name='price')
s3 = pg.State(monty, name='monty')

Initialize our network and add edges between the states. Which door Monty will choose (s3) is dependent on what door the guest chose (s1) and where the price is (s2). So, we have dependencies here.

In [366]:
bn = pg.BayesianNetwork("Solving the Monty Hall problem")
bn.add_states(s1, s2, s3)
bn.add_edge(s1, s3)
bn.add_edge(s2, s3)
bn.bake()

Here, we can play around with different combinations of given facts.

In [367]:
# Guest picks door A
beliefs = bn.predict_proba({'guest':'A'})
# Guest picks door A, Monty picks door B
#beliefs = bn.predict_proba({'guest':'A', 'monty':'B'})
# Monty picks door B, the price is behind door A
#beliefs = bn.predict_proba({'monty':'B', 'price':'A'})
# Guest picks door A, price is behind door B

Extract our beliefs from the belief network. We want to print the results and chose this simple format for convenience:  
&nbsp;&nbsp;&nbsp;&nbsp;(state, list of tuples with door name and probability)

Example:  
&nbsp;&nbsp;&nbsp;&nbsp;('guest', [('A', 0.333), ('B', 0.333), ('C', 0.333)])

In [368]:
def belief_array(belief):
    ret = []
    for door in doors:
        val = 0
        if door == belief:
            val = 1
        ret.append((door,val))
    return ret
def belief_as_tuple_list(belief):
    return belief_array(belief) if isinstance(belief, str) else list(belief.parameters[0].items())
beliefs = [(state.name, belief_as_tuple_list(belief)) for state, belief in zip (bn.states,beliefs)]

Print the probabilities

In [371]:
print('\n'.join('{}\n{}'.format(belief[0],'\n'.join('\t{}: {:.2f}'.format(x,float(y)) for x,y in belief[1])) for belief in beliefs))

guest
	A: 1.00
	B: 0.00
	C: 0.00
price
	A: 0.33
	B: 0.33
	C: 0.33
monty
	C: 0.50
	A: 0.00
	B: 0.50
